In [1]:
import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import os
import math
from sklearn.cluster import KMeans
from sklearn.metrics import precision_score
import seaborn as sns
from sklearn import metrics
import time
import pickle
import metric_learn
from sklearn.decomposition import PCA  
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import LocalOutlierFactor
import functools
import tempfile
import datetime
import tensorflow as tf
import mlflow
from mlflow.models import infer_signature
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.python.client import device_lib; print(device_lib.list_local_devices())
import tensorflow as tf
tf.config.list_physical_devices('GPU')
from utils import make_data, make_data_rev, make_label, calc_leq, leq_filter, validate, figure, figure_detail, plot_timeseries, fig_pr, auc_gs, fig_th_f

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6511909033300535279
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7798259712
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11285291128736656131
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2901478827647995270
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7798259712
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9838609608811257019
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


## Conv1D_autoencoder

In [ ]:
mlflow.autolog(log_models=False)

# 実験をアクティブ化(設定)する。ない場合は新たな実験を作成してアクティブ化する
mlflow.set_experiment("tuning_inputfilter_latent_filter")
run_name='result'
best_auc=0
with mlflow.start_run(run_name=run_name):
#     グリッドサーチ
    input_f_s, input_f_e=(10, 14)
    latent_f_s, latent_f_e=(10, 14)
    for input_f in range(input_f_s, input_f_e):
        for latent_f in range(latent_f_s, latent_f_e):
            data_dir='filtered_csv_label'
            original = pd.read_csv( os.path.join(data_dir, 'learning_data_10.csv') )
            df = pd.read_csv( os.path.join(data_dir, 'learning_data_10.csv') )
            df=df[df['day']<=9]
            test_day=[6, 7, 8, 9]
            semi_train_day=[0, 1, 2, 3, 4, 5]

            test_df=df[(df['day'].isin(test_day))].reset_index(drop=True)
            semi_train_df=df[(df['day'].isin(semi_train_day))].reset_index(drop=True)

            df_list=[]
            window=16
            for i in set(df['day']):
                lag=[] 
                lag.append(make_data_rev(semi_train_df.loc[semi_train_df['day']==i, ['original']], 'original', window))
                lag.append(semi_train_df.loc[semi_train_df['day']==i, ['day']])
                lag.append(semi_train_df.loc[semi_train_df['day']==i, ['label']])
                df_add_lag=pd.concat(lag, axis=1)
                df_list.append(df_add_lag)
            semi_train_df=pd.concat(df_list)
            semi_train_df=semi_train_df.dropna().reset_index(drop=True)
            semi_train_df.loc[:, 'convaed']=0
            semi_train_df.loc[:, 'convaed_label']=0

            # padding=sameは元の入力と同じ長さを出力がもつように入力にパディングする．
            # （※正確には，入力の大きさをstridesの大きさで単純に割ったものが出力の大きさになる）
            input_filter=input_f
            latent_filter=latent_f
            kernel_size=6
            print(f'inputfilter{input_filter}_latent_filter{latent_filter}')

            conv_autoencoder = keras.Sequential(
                [
                    layers.Conv1D(
                        filters=input_filter,
                        kernel_size=kernel_size,
                        padding="same",
                        strides=2,
                        activation="relu",
                    ),

                    layers.Conv1D(
                        filters=latent_filter,
                        kernel_size=kernel_size,
                        padding="same",
                        strides=2,
                        activation="relu",
                    ),
                    layers.Conv1DTranspose(
                        filters=latent_filter,
                        kernel_size=kernel_size,
                        padding="same",
                        strides=2,
                        activation="relu",
                    ),

                    layers.Conv1DTranspose(
                        filters=input_filter,
                        kernel_size=kernel_size,
                        padding="same",
                        strides=2,
                        activation="relu",
                    ),
                    layers.Conv1DTranspose(filters=1, kernel_size=kernel_size, padding="same"),
                ]
            )
            # テストデータとする日
            i=[1]
            label_train_day=[k for k in semi_train_day if k not in i]
            label_grant_day=i
            print(f'訓練データ：{label_train_day}')
            print(f'ラベル付与データ：{label_grant_day}日目')
            semi_train_x=semi_train_df[(semi_train_df['day'].isin(label_train_day))][semi_train_df.columns[:-4]].reset_index(drop=True)
            labeling_x=semi_train_df[(semi_train_df['day'].isin(label_grant_day))][semi_train_df.columns[:-4]].reset_index(drop=True)
            train_mean=semi_train_x['original'].mean()
            train_std=semi_train_x['original'].std()
            
             # Log the hyperparameters
            mlflow.log_params({'trainday':label_train_day, 'testday':label_grant_day, 
                               'window_size':window, 'input_filter_range':f'range({input_f_s},{input_f_e})', 
                              'latent_filter_range':f'range({latent_f_s},{latent_f_e})', 'kernel_size':kernel_size})

            #標準化
            train_data = (semi_train_x.values-train_mean)/train_std
            labeling_data = (labeling_x.values -train_mean) / train_std

            # データをモデルに入力する形状に整形し,データの型をtf.float32に変換
            train_data = tf.cast(train_data[:, :, np.newaxis], tf.float32)
            labeling_data = tf.cast(labeling_data[:, :, np.newaxis], tf.float32)

            conv_autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                                     loss='mae')

            callback = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=0,
                patience=10,
                restore_best_weights=True
            )
            log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

            history = conv_autoencoder.fit(train_data, train_data, 
                  epochs=2, 
                  batch_size=1024,
                  validation_data=(labeling_data, labeling_data),
                  callbacks=[callback, tensorboard_callback],
                  shuffle=True)

            plt.plot(history.history["loss"], label="Training Loss")
            plt.plot(history.history["val_loss"], label="Validation Loss")
            plt.legend()
            plt.show()

            reconstructions = conv_autoencoder.predict(labeling_data)
            #     異常度算出
            d= tf.keras.losses.mae(reconstructions[:, :, 0], labeling_data[:, :, 0]).numpy()
            semi_train_df.loc[semi_train_df['day'].isin(label_grant_day), 'convaed']=d
            print('------------------------------------------------------')

            test_v=semi_train_df.loc[semi_train_df['day'].isin(label_grant_day), ['original', 'label']].copy().reset_index(drop=True)
            d_test=(d-d.min())/(d.max()-d.min())

            thr_bins=10
            p_score, r_score, f_score, thresholds, thr, auc = fig_pr(test_v, d_test, thr_bins)
            f_max=f_score.max()
            p_max=p_score[np.argmax(f_score)]
            r_max=r_score[np.argmax(f_score)]
            print('\033[31m'+f'AUC：{auc}'+'\033[0m')
            print(f'理想の閾値：{thr}')
            print(f'F値（最高）：{f_max}')
            print(f'適合率（F値が最高の時）：{p_max}')
            print(f'再現率（F値が最高の時）：{r_max}')

            if best_auc<auc:
                best_auc=auc
                best_parameters = {'input_filter': input_filter, 'latent_filter':latent_filter}
                # ベストパラメータを上書き
                mlflow.log_metric("best_parameter-input_filter", best_parameters['input_filter'])
                mlflow.log_metric("best_parameter-latent_filter", best_parameters['latent_filter'])

                ################################################
                # Log the loss metric
                mlflow.log_metric("best_test_AUC", auc)
                mlflow.log_metric("best_test_Fscore", f_max)
                mlflow.log_metric("best_test_Precision_when Fscore Best", p_max)
                mlflow.log_metric("best_test_Recall_when Fscore Best", r_max)

                # 一時ディレクトリに予測結果のグラフを保存して，mlflowに送信する
                with tempfile.TemporaryDirectory()  as tmp:
                    filename = os.path.join(tmp, "predict_results.png")
                    plot_timeseries([test_v['original'].values], test_v['label'].values, d=d_test, thr=thr, mlflow=filename)
                    mlflow.log_artifact(filename, artifact_path="plot_timeseries")

            #     モデルの構造を定義しているソースコードを保存
                mlflow.log_artifact('Conv1D_autoencoder_tuning.ipynb', artifact_path="source_code")    
                # Log the model
                signature= infer_signature(labeling_data.numpy()[:2], conv_autoencoder.predict(labeling_data.numpy()[:2]))
                input_example= labeling_data.numpy()[:2]
                mlflow.tensorflow.log_model(
                    conv_autoencoder, 
                    artifact_path="Conv1D_autoencoder",
                    signature=signature,
                    registered_model_name="Conv1D_autoencoder",
                    input_example= input_example
                )

2024/01/04 14:54:00 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2024/01/04 14:54:00 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/01/04 14:54:01 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



inputfilter10_latent_filter10
訓練データ：[0, 2, 3, 4, 5]
ラベル付与データ：[1]日目


2024/01/04 14:54:04 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: 'Sequential' object has no attribute '_nested_inputs'
2024/01/04 14:54:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Arakawa\anaconda3\envs\tf291\lib\site-packages\mlflow\tensorflow\_autolog.py:52: UserWarning: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data."


Epoch 1/2
 799/2057 [==========>...................] - ETA: 5s - loss: 0.1021

In [ ]:
! mlflow gc --backend-store-uri ./mlruns

In [ ]:
! mlflow ui --port 5003 